In [4]:

# 📥 הורדת Firefox ו-Geckodriver בצורה נכונה
!apt-get update
!apt install -y wget unzip
!wget -q https://ftp.mozilla.org/pub/firefox/releases/115.0.2/linux-x86_64/en-US/firefox-115.0.2.tar.bz2
!tar xjf firefox-115.0.2.tar.bz2
!mv firefox /usr/local/firefox
!ln -s /usr/local/firefox/firefox /usr/local/bin/firefox

# 📥 הורדת Geckodriver
!wget -q https://github.com/mozilla/geckodriver/releases/download/v0.36.0/geckodriver-v0.36.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.36.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

!pip install selenium
!pip install tldextract
!pip install bs4


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [5]:

stage_3_config_dict = {
        "url_of_anchor_upper_tresh":0.6,
        "url_of_anchor_lower_tresh": 0.31,
        "link_count_html_upper_tresh":0.81,
        "link_count_html_lower_tresh": 0.13,
        "request_url_upper_tresh":0.51,
        "iframe_upper_tresh":6,
        "iframe_lower_tresh": 2,
        "js_lower_tresh":2,
        "js_upper_tresh": 6,
        "nlp_upper_tresh":0.008,
        "nlp_lower_tresh":0.003,

    }

    #for java script sus behivor
def get_high_risk_patterns():
         return [
                 r"eval\s*\(",
                 r"new\s+Function\s*\(",
                 r"document\.write\s*\(",
                 r"onmouseover\s*=",
                 r"setTimeout\s*\(\s*['\"]",
             ]
def get_medium_risk_patterns():
     return [
     r"window\.location",
     r"innerHTML\s*=",
     r"onbeforeunload",
 ]
def get_low_risk_patterns():
    return[
     r"navigator\.clipboard",
     r"XMLHttpRequest",
     r"fetch\s*\("
 ]
def get_known_safe_script_hosts():
    return [
     "cdnjs.cloudflare.com",
     "cdn.jsdelivr.net",
     "ajax.googleapis.com",
     "fonts.googleapis.com",
     "fonts.gstatic.com",
     "stackpath.bootstrapcdn.com",
     "ajax.aspnetcdn.com",
     "maxcdn.bootstrapcdn.com",
     "code.jquery.com",
     "cdn.jsdelivr.net",
     "cdn.shopify.com",
     "cdn.wix.com",
     "unpkg.com",
     "polyfill.io",
     "bootstrapcdn.com",
     "gstatic.com",
     "google.com",
     "googleapis.com",
     "microsoft.com",
     "cloudflare.com",
     "cloudfront.net",
     "fbcdn.net",
     "facebook.com",
     "yahooapis.com",
     "notion.so",
     "vercel.app",
     "netlify.app",
     "res.cloudinary.com"
 ]

def get_known_favicon_hosts():
   return ["google.com", "gstatic.com", "googleusercontent.com", "googleapis.com", "youtube.com",
                        "ytimg.com",
                        "apple.com", "microsoft.com", "office.com", "windows.com", "live.com", "microsoftonline.com",
                        "adobe.com", "typekit.net", "adobestatic.com", "facebook.com", "fbcdn.net", "instagram.com",
                        "cdninstagram.com", "twitter.com", "twimg.com",
                        "linkedin.com", "licdn.com", "pinterest.com", "pinimg.com", "reddit.com", "redditstatic.com",
                        "tumblr.com", "static.tumblr.com",
                        "fonts.googleapis.com", "fonts.gstatic.com", "ajax.googleapis.com",
                        "cloudflare.com", "cdnjs.cloudflare.com", "cdn.jsdelivr.net",
                        "cdn.shopify.com", "stackpath.bootstrapcdn.com", "ajax.aspnetcdn.com",
                        "akamaihd.net", "akamaized.net", "fastly.net", "cloudfront.net", "unpkg.com",
                        "raw.githubusercontent.com", "github.com", "github.githubassets.com",
                        "wp.com", "i0.wp.com", "i1.wp.com", "i2.wp.com",
                        "squarespace.com", "squarespace-cdn.com", "static1.squarespace.com", "shopify.com",
                        "cdn.shopify.com", "wix.com", "wixstatic.com",
                        "paypal.com", "paypalobjects.com", "ebay.com", "ebaystatic.com",
                        "amazon.com", "amazonaws.com",
                        "yahoo.com", "yimg.com", "yahooapis.com", "bootstrapcdn.com", "maxcdn.bootstrapcdn.com",
                        "jsdelivr.net", "fastly.com",
                        "googletagmanager.com", "googlesyndication.com", "doubleclick.net", "googledomains.com",
                        "firebaseio.com", "firebaseapp.com", "notion.so", "notion-static.com",
                        "netlify.app", "vercel.app", "cloudinary.com", "res.cloudinary.com"
                        ]
def get_suspicious_keywords():
     return ["login", "signin", "verify", "auth", "password", "2fa", "secure"]
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from tldextract import extract
import re
import pandas as pd
import os
import time
import logging


from re import error
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from tldextract import extract
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
import logging
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing import Process

SUSPICIOUS_WORDS_REGEX = re.compile(
    r"(log[\s\-]?in|sign[\s\-]?in|auth|user(name)?|email|phone|account|"
    r"credential|password|passcode|pin|security[\s\-]?code|credit[\s\-]?card|cvv|expiry|iban|bank)",
    re.IGNORECASE
)


lEGIT=1
sUS=0
pHISHING=-1
import pandas as pd
import numpy as np

# הדאטה שלך
data = {
    "URL": [
        "https://heatmap.com", "https://gamepedia.com",
        "https://9xflix.pink", "https://redditblog.com"
    ],
    "validity": ['safe', 'safe', 'safe', 'safe']
}

# יוצרים DataFrame
df = pd.DataFrame(data)

# מחלקים ל-4 חלקים שווים (בערך)
splits = np.array_split(df, 4)

# שומרים כל חלק לקובץ נפרד
for idx, split_df in enumerate(splits, start=1):
    output_path = f"/content/test_urls_part_{idx}.csv"
    split_df.to_csv(output_path, index=False)
    print(f"✅ Created file: {output_path}")

# מציג את הראשון כדוגמה
splits[0].head()
def safe_extract(tag, attribute):
    try:
        return tag.get(attribute,"").strip()
    except Exception:
        return ""


✅ Created file: /content/test_urls_part_1.csv
✅ Created file: /content/test_urls_part_2.csv
✅ Created file: /content/test_urls_part_3.csv
✅ Created file: /content/test_urls_part_4.csv


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:

def element_extraction_from_html(soup_html:BeautifulSoup,tag:str=None,attribute=None)->list:
   #generic function to extract <tag> and its attribute like <link> "href"
    if not tag:
        return []
    if attribute:
       #change to this ->> soup_html.find_all(tag, atribute=True)
        return soup_html.find_all(tag,**{attribute:True})
    else:
        return soup_html.find_all(tag)

#done

def normalize_domain(url:str):
    parts_of_url = extract(url)
    if not (parts_of_url.domain and parts_of_url.suffix):
        return None
    domain=f"{parts_of_url.domain}.{parts_of_url.suffix}"
    return domain
''' specific  function: check the html element that competable with each tag that indicates for a phishing site '''



#####################1-favicon#############################

def has_icon_func(icon_links:list)->int:
    has_icon= 1 if len(icon_links) > 0 else 0
    return has_icon

def favicon_check(link_tag:list, base_domain:str) -> tuple:

    icon_links = []
    expected_domain = base_domain.lower()
    favicon_domain_not_the_same=0
    favicon_endwith=0
    #filter only link tags related to favicon
    for link in link_tag:
        rel = link.get("rel", []) #return all relations to the page (rel="something) is in the element <link>)
        if any("icon" in r.lower() for r in rel):
            icon_links.append(link)
    has_icon = has_icon_func(icon_links)

    for link in icon_links:
        href = link["href"]
        parsed =normalize_domain(href)

        if not parsed:
            continue

        domain = parsed.lower()
        if any(safe in domain for safe in get_known_favicon_hosts()):
            continue

        if not (domain == expected_domain):
            favicon_domain_not_the_same+=1

        # Check for acceptable file extensions in the requested url path.
        if not (href.endswith(".ico") or href.endswith(".png") or href.endswith(".gif")):
            favicon_endwith+=1

    return has_icon,favicon_domain_not_the_same,favicon_endwith


#####################2-anchor#############################
# <a> tag ,attr - href=

def extract_url_of_anchor_feature(a_list : list,base_domain) -> tuple:

    total = int(len(a_list))
    anchor_tags_count = 1 if total>0 else 0
    anchor_empty_href_count = 0
    anchor_domain_not_the_same = 0

    for tag in a_list:
        href =safe_extract(tag,"href")

        if (not href) or href.strip() in ["#", "javascript:void(0);", "javascript:"]:
            anchor_empty_href_count += 1
            continue

        href_domain = normalize_domain(href)
        if not href_domain:
            continue
        if  href_domain != base_domain:
            anchor_domain_not_the_same += 1

    return anchor_tags_count,anchor_empty_href_count,anchor_domain_not_the_same


#####################3-link_count#############################
# <script> attr-> src =,<meta> attr - content,<links> tag ,attr - href=
#here i divided the big function into 4 smaller ones for each component
def count_external_script_src(script_list:list, base_domain: str)->tuple:
    external_script_count=0
    sus_words_in_script=0

    for tag in script_list:
        value=safe_extract(tag, "src")
        if not value:
            continue
        domain = normalize_domain(value)
        if not domain:
            continue
        if domain != base_domain:
            external_script_count += 1
        text = " ".join((tag.string or ""))
        if not text:
            continue
        matches = SUSPICIOUS_WORDS_REGEX.findall(text.lower())
        sus_words_in_script +=len(matches)

    return external_script_count , sus_words_in_script

def count_external_meta_content(meta_list: list, base_domain: str) -> tuple:
    external_meta_count = 0
    sus_words_in_meta=0
    for tag in meta_list:
        value = safe_extract(tag, "content")
        if not value:
            continue
        domain = normalize_domain(value)
        if not domain:
            continue
        if domain != base_domain:
            external_meta_count += 1
        text = " ".join(value)
        if not text:
            continue
        matches = SUSPICIOUS_WORDS_REGEX.findall(text.lower())
        sus_words_in_meta += len(matches)
    return external_meta_count,sus_words_in_meta


def count_external_link_href(link_list: list, base_domain: str) -> int:
    external_link_count = 0
    for tag in link_list:
        value = safe_extract(tag, "src")
        if not value:
            continue
        domain = normalize_domain(value)
        if not domain:
            continue
        if domain != base_domain:
            external_link_count += 1

    return external_link_count


def link_count_in_html(extern_links:int,extern_meta:int,extern_script:int) -> int:
    total_extern_links=extern_links +extern_meta +extern_script
    return total_extern_links


#####################4-request_url#############################
# tags--->    < img,source,audio,video,embed,iframe >, attr-> src

def extract_request_url_feature(resources_elements_list: list, base_domain: str) -> tuple:
    total_resources = len(resources_elements_list)
    external_count = 0

    for tag in resources_elements_list:
        src = safe_extract(tag, "src")
        if not src:
            continue
        domain = normalize_domain(src)
        if not domain:
            continue
        if domain != base_domain:
            external_count += 1

    return total_resources, external_count


#####################5-sfh#############################
# <form> tag ,attr - action/nothing=

def extract_sfh_feature(form_list: list, base_domain: str) -> tuple:
    sfh_count=len(form_list)
    sfh_action_is_blank = 0
    sfh_domain_not_the_same=0
    password_in_sfh=0
    has_suspicious_words=0
    for form in form_list:
        action = safe_extract(form, "action")
        if not action or action.lower() in ["about:blank", "#", ""]:
            sfh_action_is_blank += 1
        else:
            action_domain = normalize_domain(action)
            if action_domain and action_domain != base_domain:
                sfh_domain_not_the_same += 1

        inputs = form.find_all("input")
        for i in inputs:
            t = i.get("type", "").lower()
            n = i.get("name", "").lower()
            if t == "password":
                password_in_sfh += 1
            if any(w in n for w in get_suspicious_keywords()):
                has_suspicious_words += 1

    return  sfh_count,sfh_action_is_blank,sfh_domain_not_the_same,password_in_sfh,has_suspicious_words

#####################6-iframe#############################
# <iframe> tag ,attr - src/srcdoc=

def extract_iframe_feature_src(frame_src_list: list, base_domain: str) -> tuple:

    iframe_src_count=0
    iframe_src_style_hidden=0
    iframe_src_size=0
    iframe_src_domain_not_the_same=0
    iframe_no_src_sendbox=0
    try:
        for iframe in frame_src_list:

            src = iframe.get("src", "").strip().lower()
            if src:
                iframe_src_count+=1

            style = iframe.get("style", "").lower()
            width = iframe.get("width", "").strip()
            height = iframe.get("height", "").strip()
            iframe_domain = normalize_domain(src) if src else ""

            is_external = iframe_domain and iframe_domain != base_domain

            if any(x in src for x in ["ads", "analytics", "pixel", "tracker", "doubleclick"]):
                continue

            if "display:none" in style or "visibility:hidden" in style:
                iframe_src_style_hidden += 1

            if width == "0" or height == "0":
                iframe_src_size += 1

            if is_external:
                iframe_src_domain_not_the_same += 1

            if not iframe.has_attr("sandbox"):
                iframe_no_src_sendbox += 1
    except Exception as e:
        return 0,0,0,0,0
    return   iframe_src_count, iframe_src_style_hidden,  iframe_src_size, iframe_src_domain_not_the_same, iframe_no_src_sendbox

def extract_iframe_feature_srcdoc(iframe_list: list, base_domain: str) -> tuple:
    iframe_srcdoc_count = 0
    iframe_src_doc_hidden = 0
    iframe_srcdoc_js_existence = 0
    iframe_srcdoc_sus_words= 0

    try:
        for iframe in iframe_list:

            srcdoc = iframe.get("srcdoc", "").strip().lower()

            if srcdoc:
                iframe_srcdoc_count+=1
                clean_srcdoc_text = BeautifulSoup(srcdoc, "html.parser").get_text().lower()

                if SUSPICIOUS_WORDS_REGEX.search(clean_srcdoc_text):
                    iframe_srcdoc_sus_words += 1
                if "<script" in srcdoc or "javascript:" in srcdoc:
                    iframe_srcdoc_js_existence += 1
                if "display:none" in srcdoc or "visibility:hidden" in srcdoc:
                    iframe_src_doc_hidden += 1

        return iframe_srcdoc_count,  iframe_src_doc_hidden,   iframe_srcdoc_js_existence , iframe_srcdoc_sus_words
    except Exception as e:
        return iframe_srcdoc_count,  iframe_src_doc_hidden,   iframe_srcdoc_js_existence , iframe_srcdoc_sus_words

def total_iframe_src_n_doc(src_count:int,srcdoc_count:int)->int:
    return src_count+srcdoc_count

#####################7-suspicious_js#############################

def detect_suspicious_js_behavior(soup: BeautifulSoup, base_domain: str) -> tuple:
    score = 0
    inline_scripts_count=0
    high_risk_patterns_count=0
    medium_risk_patterns_count=0
    low_risk_patterns_count=0
    sus_js_domain_not_the_same=0

    try:
        inline_scripts = soup.find_all("script", src=False)
    except Exception:
        return inline_scripts_count,high_risk_patterns_count,medium_risk_patterns_count,low_risk_patterns_count,sus_js_domain_not_the_same
    inline_scripts_count=len(inline_scripts)

    for script in inline_scripts:
        try:
            content = script.get_text().strip().lower()
        except Exception:
            content = ""

        for pattern in get_high_risk_patterns():
            if re.search(pattern, content):
                high_risk_patterns_count += 1

        for pattern in get_medium_risk_patterns():
            if re.search(pattern, content):
                medium_risk_patterns_count += 1

        for pattern in get_low_risk_patterns():
            if re.search(pattern, content):
                low_risk_patterns_count += 1

    try:
        external_scripts = soup.find_all("script", src=True)
        for script in external_scripts:
            src = safe_extract(script, "src")
            domain = normalize_domain(src)
            if domain and domain != base_domain and domain not in get_known_safe_script_hosts():
                sus_js_domain_not_the_same += 1

        return inline_scripts_count,    high_risk_patterns_count,    medium_risk_patterns_count,    low_risk_patterns_count,   sus_js_domain_not_the_same

    except Exception:
        return inline_scripts_count,    high_risk_patterns_count,    medium_risk_patterns_count,    low_risk_patterns_count,   sus_js_domain_not_the_same


#####################8-nlp#############################

def nlp_based_phishing_text_check(soup: BeautifulSoup) -> int:
    text = soup.get_text(strip=True).lower()

    matches = SUSPICIOUS_WORDS_REGEX.findall(text)

    return len(matches)


#####################9-analyze_textual_tags#############################

def analyze_textual_tags(count_script:int,count_meta:int) -> int:
    return count_script + count_meta


##### to stage 3 full loaded html
####################10-dynamic_script#############################
def detect_dynamic_script_injection(driver: webdriver) -> int:
    try:
        injected_scripts = driver.execute_script("""
            return [...document.scripts].filter(s => s.src || s.innerText.length > 0).length;
        """)
        return injected_scripts
    except Exception as e:
        return sUS

#####################11-auto_redirect#############################
############## after all html was loaded
# this function is blocking because the waiting for full upload of the page ,
def detect_auto_redirect(driver: webdriver, base_domain: str, timeout: float = 3.0) ->tuple:
    meta_equiv=0
    window_or_replace_redirect=0
    autoredirect_different_domain=0
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )

        page_source = driver.page_source.lower()
    except Exception:
        return meta_equiv,  window_or_replace_redirect, autoredirect_different_domain
    try:

        if re.search(r'<meta\s+http-equiv\s*=\s*["\']?refresh["\']?', page_source, re.IGNORECASE):
            meta_equiv=1
    except Exception:
        pass
    try:
        if re.search(r'(window\.)?location\.(href|replace)', page_source):
            window_or_replace_redirect=1
    except Exception:
        pass
    try:
        final_url = driver.current_url

        if normalize_domain(final_url) != base_domain:
            autoredirect_different_domain=1

    except Exception:
        pass

    return meta_equiv, window_or_replace_redirect, autoredirect_different_domain



#####################12-login_form_visibility#############################
def check_login_form_visibility(driver: webdriver) -> int:
    hidden=0
    try:
        script = """
        var forms = document.getElementsByTagName('form');
        for (var i = 0; i < forms.length; i++) {
            var style = window.getComputedStyle(forms[i]);
            if (style.display === 'none' || style.visibility === 'hidden' ||
                forms[i].offsetWidth === 0 || forms[i].offsetHeight === 0) {
                return true;
            }
        }
        return false;
        """
        hidden = driver.execute_script(script)
    except Exception:
        pass
    return int(hidden)

#####################13-onmouseover#############################
def detect_onmouseover_in_dom(soup: BeautifulSoup) -> tuple:
    tags_with_onmouseover_count=0
    suspicious_script_detected=0
    try:
        tags_with_onmouseover = soup.find_all(attrs={"onmouseover": True})
        tags_with_onmouseover_count=  len(tags_with_onmouseover)
        inline_scripts = soup.find_all("script", src=False)
        suspicious_script_detected = sum(1 for script in inline_scripts if "onmouseover" in (script.string or "").lower())
    except Exception:
        pass

    return suspicious_script_detected,tags_with_onmouseover_count

#####################14-right_click_block#############################
### when checking in browser return to original function because it is very sus
def detect_right_click_block(soup: BeautifulSoup) -> tuple:

    suspicious_script_count=0
    contextmenu_tags_count=0
    try:
        contextmenu_tags = soup.find_all(attrs={"oncontextmenu": True})
        contextmenu_tags_count=len(contextmenu_tags)
    except Exception:
        pass
    try:

        inline_scripts = soup.find_all("script", src=False)
        for script in inline_scripts:
            content = (script.string or "").lower()

            if not content:
                continue

            if re.search(r"event\s*\.\s*button\s*==\s*2", content) or "contextmenu" in content:
                suspicious_script_count +=1
    except Exception:
        pass
    return suspicious_script_count,contextmenu_tags_count

##############################################end###################################


def find_html_features_separated(soup: BeautifulSoup, url: str, feature_type: str, driver: webdriver):
    domain = normalize_domain(url)
    elements = []

    if feature_type == "favicon_check":
        elements = element_extraction_from_html(soup, tag="link", attribute="href")
        return favicon_check(elements, domain)

    elif feature_type == "url_anchor":
        elements = element_extraction_from_html(soup, tag="a", attribute="href")
        return extract_url_of_anchor_feature(elements, domain)

    elif feature_type == "links_in_tags":
        meta_elements = element_extraction_from_html(soup, tag="meta", attribute="content")
        script_elements = element_extraction_from_html(soup, tag="script", attribute="src")
        link_elements = element_extraction_from_html(soup, tag="link", attribute="href")
        extern_meta, sus_words_meta = count_external_meta_content(meta_elements, domain)
        extern_script, sus_words_script = count_external_script_src(script_elements, domain)
        extern_links = count_external_link_href(link_elements, domain)
        total_extern = link_count_in_html(extern_links, extern_meta, extern_script)
        return extern_meta, sus_words_meta, extern_script, sus_words_script, extern_links, total_extern

    elif feature_type == "request_sources_from_diff_url":
        for tag in ["img", "source", "audio", "video", "embed", "iframe"]:
            elements += element_extraction_from_html(soup, tag=tag, attribute="src")
        return extract_request_url_feature(elements, domain)

    elif feature_type == "sfh":
        elements = element_extraction_from_html(soup, tag="form")
        return extract_sfh_feature(elements, domain)

    elif feature_type == "iframe":
        iframe_elements = element_extraction_from_html(soup, tag="iframe")
        src_features = extract_iframe_feature_src(iframe_elements, domain)
        srcdoc_features = extract_iframe_feature_srcdoc(iframe_elements, domain)
        total_iframes = total_iframe_src_n_doc(src_features[0], srcdoc_features[0])
        return src_features + srcdoc_features + (total_iframes,)

    elif feature_type == "suspicious_js":
        return detect_suspicious_js_behavior(soup, domain)

    elif feature_type == "nlp_text":
        return (nlp_based_phishing_text_check(soup),)

    elif feature_type == "analyze_textual_tags":
        meta_elements = element_extraction_from_html(soup, tag="meta", attribute="content")
        script_elements = element_extraction_from_html(soup, tag="script", attribute="src")
        count_meta, _ = count_external_meta_content(meta_elements, domain)
        count_script, _ = count_external_script_src(script_elements, domain)
        return (analyze_textual_tags(count_script, count_meta),)

    elif feature_type == "detect_dynamic_script_injection":
        return (detect_dynamic_script_injection(driver),)

    elif feature_type == "detect_auto_redirect":
        return detect_auto_redirect(driver, domain)

    elif feature_type == "check_login_form_visibility":
        return (check_login_form_visibility(driver),)

    elif feature_type == "detect_onmouseover_in_dom":
        return detect_onmouseover_in_dom(soup)

    elif feature_type == "detect_right_click_block":
        return detect_right_click_block(soup)

    return ()
def wait_for_initial_html(driver, min_length=500, timeout=5.0) -> str:
    """
    Waits until there is enough initial HTML content.
    Returns the HTML once ready, or the best available after timeout.
    """
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            ready_state = driver.execute_script("return document.readyState;")
            if ready_state not in ["interactive", "complete"]:
                time.sleep(0.05)
                continue

            html = driver.execute_script("return document.documentElement.outerHTML;")
            if html and len(html) >= min_length:
                return html
        except Exception:
            pass

        time.sleep(0.05)

    # fallback
    try:
        html = driver.execute_script("return document.documentElement.outerHTML;")
    except Exception:
        html = ""
    return html

def test_headless_browser_firefox(url: str, mode: str = "full") -> tuple:
    """
    mode: "initial" -> fetch initial enough HTML
          "full"    -> wait for fully loaded HTML
    """
    driver = None
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'http://' + url

        ffx_options = Options()
        ffx_options.add_argument("--headless")
        driver = webdriver.Firefox(options=ffx_options)
        driver.get(url)

        if mode == "full":

            # 🔵 מצב FULL → לחכות שהדף יטען לגמרי
            WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
            html = driver.page_source
        elif mode == "initial":
            # 🟢 מצב INITIAL → לחכות שיהיה מספיק HTML ראשוני
            html = wait_for_initial_html(driver, min_length=500, timeout=5.0)
        else:
            raise ValueError("Unknown mode. Use 'initial' or 'full'.")

        if not html or html.strip() == "":
            logging.error(f"[ERROR] Empty page source for {url}")
            return None, None

        return html, driver

    except Exception as e:
        logging.error(f"❌ [ERROR] Failed to load {url}: {e}")
        return None, None

    finally:
        if driver is not None:
            try:
                driver.quit()
            except Exception:
                pass



# Set logging level to DEBUG
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s')

def extract_stage3_features_debug_separated(input_csv_path, output_csv_path,pid):
    df = pd.read_csv(input_csv_path)
    total = len(df)
    results = []

    for index, row in df.iterrows():
        url = row['URL']
        label = row.get('label', 0)
        print(f"proc id:{pid}\n🔍 [{index+1}/{total}] Processing URL: {url}")

        html, driver = test_headless_browser_firefox(url,"full")    #change here to "full" or "initial"
        if html is None or driver is None:
            logging.error(f"[ERROR] Failed to load page: {url}")
            continue

        try:
            soup = BeautifulSoup(html, "html.parser")
            features = [url]

            feature_types = [
                "favicon_check",
                "url_anchor",
                "links_in_tags",
                "request_sources_from_diff_url",
                "sfh",
                "iframe",
                "suspicious_js",
                "nlp_text",
                "analyze_textual_tags",
                "detect_dynamic_script_injection",
                "detect_auto_redirect",
                "check_login_form_visibility",
                "detect_onmouseover_in_dom",
                "detect_right_click_block"
            ]

            for feature_type in feature_types:
                try:
                    result = find_html_features_separated(soup, url, feature_type, driver)
                    features.extend(result)
                except Exception as e:
                    logging.error(f"[ERROR] {feature_type} failed for {url} → {e}")
                    features.extend([-999])

            features.append(label)
            results.append(features)
        finally:
            if driver is not None:
                 try:
                     driver.quit()
                 except Exception:
                     pass

    headers = [
        "url",
        # favicon
        "has_icon", "favicon_diff_domain", "favicon_invalid_ext",
        # anchor
        "anchor_tags_present", "anchor_empty_href", "anchor_diff_domain",
        # links
        "meta_external", "meta_sus_words", "script_external", "script_sus_words", "link_external", "total_external",
        # request sources
        "total_resources", "external_resources",
        # sfh
        "sfh_total_forms", "sfh_blank_action", "sfh_diff_domain", "sfh_password_inputs", "sfh_suspicious_inputs",
        # iframe src
        "iframe_src_count", "iframe_src_hidden", "iframe_src_size", "iframe_src_diff_domain", "iframe_src_no_sandbox",
        # iframe srcdoc
        "iframe_srcdoc_count", "iframe_srcdoc_hidden", "iframe_srcdoc_scripts", "iframe_srcdoc_sus_words",
        # total iframes
        "total_iframes",
        # suspicious_js
        "inline_scripts", "high_risk_patterns", "medium_risk_patterns", "low_risk_patterns", "sus_js_diff_domain",
        # nlp
        "nlp_suspicious_words",
        # analyze_textual_tags
        "analyze_textual_sus_words",
        # dynamic script injection
        "dynamic_scripts_count",
        # auto redirect
        "meta_refresh_redirect", "window_location_redirect", "final_url_diff_domain",
        # hidden login forms
        "hidden_forms_count",
        # onmouseover
        "onmouseover_scripts", "onmouseover_tags",
        # right click block
        "right_click_scripts", "right_click_tags",
        # label
        "label"
        ]


    print("updated")
        # Save to CSV
    pd.DataFrame(results, columns=headers).to_csv(output_csv_path, index=False)
    logging.info(f"\n✅ Finished. CSV saved to: \033[92m{output_csv_path}\033[0m")





def process_single_file(input_csv_path, output_csv_path,pid):
    try:
        logging.info(f"🚀 Starting pid= {pid} for {input_csv_path}")
        extract_stage3_features_debug_separated(input_csv_path, output_csv_path,pid)
        logging.info(f"✅ Finished pid={pid} for {input_csv_path}")
    except Exception as e:
        logging.error(f"❌ Error in pid={pid} processing {input_csv_path}: {e}")

def main_parallel():

    files = [
        ("/content/test_urls_part_1.csv", "/content/test1_urls_part_1.csv","1"),
        ("/content/test_urls_part_2.csv", "/content/test1_urls_part_2.csv","2"),
        ("/content/test_urls_part_3.csv", "/content/test1_urls_part_3.csv","3"),
         ("/content/test_urls_part_4.csv", "/content/test1_urls_part_4.csv","4")
    ]


    total_files = len(files)
    print(f"🚀 Starting sequential processing of {total_files} files...\n")

    for idx, (input_csv, output_csv, id) in enumerate(files, 1):
        print(f"🔵 [{idx}/{total_files}] Processing file ID={id}")
        try:
            process_single_file(input_csv, output_csv, id)
            print(f"✅ Finished processing file ID={id}\n")
        except Exception as e:
            print(f"❌ Error processing file ID={id}: {e}\n")

    print("🎯 All files processed successfully!\n")

main_parallel()